# Volume 3: Discrete Hidden Markov Models
    Bryant McArthur
    Math 405
    February 14

In [1]:
import numpy as np
import copy
import string
import codecs

In [2]:
class HMM(object):
    """
    Finite state space hidden markov model.
    """
    
    # Problem 1
    def __init__(self):
        """
        Initialize an HMM with parameters A, B, and pi, initialized as 
        None objects.
        """
        self.A = None
        self.B = None
        self.pi = None
    
    
    # Problem 2
    def _forward(self, z):
        """
        Compute the scaled forward probability matrix and scaling factors.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        
        Returns
        -------
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix
        
        c : ndarray of shape (T,)
            The scaling factors c = [c_0, c_1, ..., c_{T-1}]
        """
        alpha = np.empty((len(z),len(self.pi)))
        
        alpha0 = self.pi*self.B[z[0]]
        c = [1/np.sum(alpha0)]
        alpha[0] = c[0]*alpha0
        
        # Forward loop
        for t in range(1, len(z)):
            for i in range(len(self.pi)):
                alpha[t,i] = np.sum([alpha[t-1,j]*self.A[i,j]*self.B[z[t],i] for j in range(len(self.pi))])
            c.append(1/np.sum(alpha[t]))
            alpha[t] = c[t]*alpha[t]
            
        return alpha, c
        
    
    
    # Problem 3
    def _backward(self, z, c):
        """
        Compute the scaled backward probability matrix.
        
        Parameters
        ----------
        z : ndarray of shape (T,) 
            The sequence of observations
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix
        """
        beta = np.empty((len(z),len(self.pi)))
        beta[-1] = np.ones(len(self.pi))*c[-1]
        
        # Backward loop
        for t in range(len(z)-2, -1, -1):
            beta[t] = [np.sum([self.A[i,j]*beta[t+1,i]*self.B[z[t+1],i] for i in range(len(self.pi))]) for j in range(len(self.pi))]
            beta[t] *= c[t]
                
        return beta
        
    
    
    # Problem 4
    def _xi(self, z, alpha, beta, c):
        """
        Compute the xi and gamma probabilities.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        alpha : ndarray of shape (T, n)
            The scaled forward probability matrix from the forward pass
        beta : ndarray of shape (T, n)
            The scaled backward probability matrix from the backward pass
        c : ndarray of shape (T,)
            The scaling factors from the forward pass

        Returns
        -------
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        T = len(z)
        n = len(self.pi)
        eps = np.empty((T-1, n, n))
        gamma = np.empty((T,n))
        
        # Epsilon
        for t in range(T-1):
            eps[t] = [[alpha[t,i]*self.A[j,i]*self.B[z[t+1],j]*beta[t+1,j] for j in range(n)] for i in range(n)]
        
        # Gamma
        for t in range(T):
            gamma[t] = np.multiply(alpha[t], beta[t])/c[t]
        
        return eps, gamma
            
    
    # Problem 5
    def _estimate(self, z, xi, gamma):
        """
        Estimate better parameter values and update self.pi, self.A, and
        self.B in place.

        Parameters
        ----------
        z : ndarray of shape (T,)
            The observation sequence
        xi : ndarray of shape (T-1, n, n)
            The xi probability array
        gamma : ndarray of shape (T, n)
            The gamma probability array
        """
        
        T = len(z)
        n = len(self.pi)
        
        # Update Parameters
        self.pi = gamma[0]
        self.A = np.sum(xi,axis=0).T/np.sum(gamma[:-1,:],axis=0)
        
        B = np.zeros_like(self.B)
        for i in range(self.B.shape[0]):
            B[i] = np.sum([gamma[t]*int(z[t]==i) for t in range(T)], axis=0)
            
        self.B = B / np.sum(gamma, axis=0)
        
        return self

    
    # Problem 6
    def fit(self, z, pi, A, B, max_iter=100, tol=1e-5):
        """
        Fit the model parameters to a given observation sequence.

        Parameters
        ----------
        z : ndarray of shape (T,)
            Observation sequence on which to train the model.
        pi : Stochastic ndarray of shape (n,)
            Initial state distribution
        A : Stochastic ndarray of shape (n, n)
            Initial state transition matrix
        B : Stochastic ndarray of shape (m, n)
            Initial state observation matrix
        max_iter : int
            The maximum number of iterations to take
        tol : float
            The convergence threshold for change in log-probability
        """
        # initialize self.pi, self.A, self.B
        self.pi = pi
        self.A = A
        self.B = B
        
        # Initial log prob
        alpha, c = self._forward(z)
        old_prob = -np.log(c).sum()
        
        # run the iteration
        for i in range(max_iter):
            alpha, c = self._forward(z)
            beta = self._backward(z, c)
            xi, gamma = self._xi(z, alpha, beta, c)
            self._estimate(z, xi, gamma)
            alpha, c = self._forward(z)
            new_prob = -np.log(c).sum()
            
            # Check Convergence
            if abs(new_prob - old_prob) < tol:
                break
            else:
                old_prob = new_prob
                continue
            
        return self

In [68]:
pi = np.array([.6,.4])
A = np.array([[.7,.4],[.3,.6]])
B = np.array([[.1,.7],[.4,.2],[.5,.1]])
z = np.array([0,1,0,2])

# Verify
h = HMM()
h.pi = pi
h.A = A
h.B = B
alpha, c = h._forward(z)
print(-np.log(c).sum())

-4.642913590898749


In [69]:
beta = h._backward(z, c)
print(beta)

[[3.1361635  2.89939354]
 [2.86699344 4.39229044]
 [3.898812   2.66760821]
 [3.56816483 3.56816483]]


In [70]:
xi, gamma = h._xi(z, alpha, beta, c)
print(xi)
print(gamma)

[[[0.14166321 0.0465066 ]
  [0.37776855 0.43406164]]

 [[0.17015868 0.34927307]
  [0.05871895 0.4218493 ]]

 [[0.21080834 0.01806929]
  [0.59317106 0.17795132]]]
[[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]]


In [71]:
h._estimate(z,xi,gamma)
print(h.pi)
print(h.A)
print(h.B)

[0.18816981 0.81183019]
[[0.55807991 0.49898142]
 [0.44192009 0.50101858]]
[[0.23961928 0.70056364]
 [0.29844534 0.21268397]
 [0.46193538 0.08675238]]


In [3]:
def vec_translate(a, my_dict):
    # translate numpy array from symbols to state numbers or vice versa
    return np.vectorize(my_dict.__getitem__)(a)

def prep_data(filename):
    # Get the data as a single string
    with codecs.open(filename, encoding='utf-8') as f:
        data=f.read().lower()  # and convert to all lower case

    # remove punctuation and newlines
    remove_punct_map = {ord(char): 
                        None for char in string.punctuation+"\n\r"}
    data = data.translate(remove_punct_map)

    # make a list of the symbols in the data
    symbols = sorted(list(set(data)))

    # convert the data to a NumPy array of symbols
    a = np.array(list(data))

    # make a conversion dictionary from symbols to state numbers
    symbols_to_obsstates = {x:i for i,x in enumerate(symbols)}

    # convert the symbols in a to state numbers
    obs_sequence = vec_translate(a,symbols_to_obsstates)

    return symbols, obs_sequence

## Problem 7

Train a HMM on `declaration.txt`. Use `N=2` states and `M=len(set(obs))=27` observation values ($26$ lower case characters and $1$ whitespace character), and run for $150$ iterations with the deffault value for `tol`.

Once the learning algorithm converges, analyze the state observation matrix $B$. Note which rows correspond to the largest and smallest probability values in each column of $B$, and check the corresponding characters. The HMM should have detected a vowel state and a consonant state.

In [ ]:
symbols, obs = prep_data('declaration.txt')
M = len(set(obs))
N = 2
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

#fit the hmm
hmm = HMM()
hmm.fit(obs, pi, A, B, max_iter=150)

In [75]:
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}"
                .format(symbols[i], hmm.B[i,0], hmm.B[i,1]))
print('\n', 'The first column is the vowel state, the second column is the constant state')

 , 0.3233, 0.0180
a, 0.1265, 0.0000
b, 0.0000, 0.0234
c, 0.0000, 0.0454
d, 0.0000, 0.0621
e, 0.2278, 0.0000
f, 0.0000, 0.0444
g, 0.0000, 0.0321
h, 0.0048, 0.0816
i, 0.1191, 0.0000
j, 0.0000, 0.0039
k, 0.0004, 0.0030
l, 0.0000, 0.0562
m, 0.0000, 0.0355
n, 0.0000, 0.1191
o, 0.1356, 0.0004
p, 0.0000, 0.0340
q, 0.0000, 0.0015
r, 0.0000, 0.1048
s, 0.0000, 0.1179
t, 0.0000, 0.1578
u, 0.0554, 0.0000
v, 0.0000, 0.0182
w, 0.0000, 0.0239
x, 0.0000, 0.0022
y, 0.0071, 0.0134
z, 0.0000, 0.0010

 The first column is the vowel state, the second column is the constant state


## Problem 8

Repeat the previous calculation with $3$ hidden states and again with $4$ hidden states. Interpret/explain your results.

In [4]:
# 3 hidden states
symbols, obs = prep_data('declaration.txt')
M = len(set(obs))
N = 3
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

# fit the hmm
hmm = HMM()
hmm.fit(obs, pi, A, B, max_iter=150)

In [6]:
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}"
                .format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2]))

 , 0.1258, 0.0307, 0.3104
a, 0.0695, 0.0000, 0.1101
b, 0.0278, 0.0146, 0.0000
c, 0.0375, 0.0399, 0.0000
d, 0.0075, 0.0847, 0.0009
e, 0.0000, 0.0000, 0.2780
f, 0.0148, 0.0545, 0.0000
g, 0.0080, 0.0413, 0.0000
h, 0.0000, 0.1168, 0.0083
i, 0.0320, 0.0000, 0.1249
j, 0.0000, 0.0058, 0.0000
k, 0.0006, 0.0040, 0.0006
l, 0.0312, 0.0601, 0.0001
m, 0.0159, 0.0407, 0.0000
n, 0.1811, 0.0457, 0.0000
o, 0.0831, 0.0000, 0.1131
p, 0.0295, 0.0289, 0.0000
q, 0.0000, 0.0022, 0.0000
r, 0.0596, 0.1112, 0.0000
s, 0.0792, 0.1125, 0.0035
t, 0.1352, 0.1351, 0.0000
u, 0.0410, 0.0000, 0.0415
v, 0.0000, 0.0267, 0.0000
w, 0.0187, 0.0217, 0.0000
x, 0.0021, 0.0018, 0.0000
y, 0.0000, 0.0196, 0.0087
z, 0.0000, 0.0014, 0.0000


First state is hard consonants

Second State is soft consonants

Third State is vowels

In [78]:
# 4 hidden states
N = 4
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

# fit the hmm
hmm = HMM()
hmm.fit(obs, pi, A, B, max_iter=150, tol=1e-5)

In [79]:
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}, {4:0.4f}"
                .format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2], hmm.B[i,3]))
print('\n', 'The first column is the consonant state, the second column is the vowel state, the third column is'
     ' , and the fourth column is')

 , 0.5546, 0.0000, 0.0000, 0.0000
a, 0.0517, 0.0254, 0.0027, 0.1322
b, 0.0000, 0.0000, 0.0000, 0.0378
c, 0.0000, 0.0000, 0.0031, 0.0703
d, 0.0000, 0.0000, 0.1106, 0.0000
e, 0.0990, 0.0071, 0.1694, 0.0944
f, 0.0000, 0.2554, 0.0001, 0.0000
g, 0.0000, 0.0000, 0.0482, 0.0080
h, 0.0954, 0.0000, 0.0411, 0.0131
i, 0.1067, 0.0000, 0.0000, 0.0797
j, 0.0000, 0.0000, 0.0070, 0.0000
k, 0.0000, 0.0000, 0.0033, 0.0026
l, 0.0002, 0.0000, 0.0332, 0.0604
m, 0.0000, 0.0183, 0.0201, 0.0339
n, 0.0000, 0.0042, 0.0600, 0.1365
o, 0.0188, 0.3382, 0.0000, 0.0919
p, 0.0000, 0.0000, 0.0000, 0.0549
q, 0.0000, 0.0000, 0.0026, 0.0000
r, 0.0164, 0.1447, 0.0582, 0.0606
s, 0.0019, 0.0000, 0.1697, 0.0346
t, 0.0000, 0.2031, 0.2182, 0.0000
u, 0.0507, 0.0000, 0.0000, 0.0362
v, 0.0000, 0.0000, 0.0000, 0.0294
w, 0.0000, 0.0000, 0.0194, 0.0210
x, 0.0032, 0.0000, 0.0000, 0.0006
y, 0.0013, 0.0035, 0.0331, 0.0000
z, 0.0000, 0.0000, 0.0000, 0.0016

 The first column is the consonant state, the second column is the vowel state, t

*Edit to the above because I don't want to rerun it.

The first state is the space

The second state is soft consonants

The third state is hard consonants

The fourth state is the vowels

## Problem 9

Repeat the same calculation with `WarAndPeace.txt` for $2$ and $3$ hidden states. Interpret/explain your results. Which Cyrillic characters appear to be vowels?

In [80]:
# 2 hidden states
symbols, obs = prep_data('WarAndPeace.txt')
M = len(set(obs))
N = 2
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

# fit the hmm
hmm = HMM()
hmm.fit(obs, pi, A, B, max_iter=150)

In [87]:
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}"
                .format(symbols[i], hmm.B[i,0], hmm.B[i,1]))
print('\n', 'The first column is more the consonant state, \
      \nthe second column is more the vowel state, \nbut there is overlap')


 , 0.1667, 0.1623
а, 0.1813, 0.0043
б, 0.0069, 0.0199
в, 0.0364, 0.0411
г, 0.0043, 0.0258
д, 0.0120, 0.0298
е, 0.0008, 0.1076
ж, 0.0062, 0.0097
з, 0.0407, 0.0000
и, 0.0771, 0.0395
й, 0.0004, 0.0141
к, 0.0696, 0.0069
л, 0.0701, 0.0272
м, 0.0157, 0.0272
н, 0.0791, 0.0462
о, 0.0049, 0.1504
п, 0.0025, 0.0357
р, 0.0183, 0.0463
с, 0.0487, 0.0380
т, 0.0182, 0.0638
у, 0.0220, 0.0245
ф, 0.0033, 0.0000
х, 0.0020, 0.0094
ц, 0.0043, 0.0022
ч, 0.0058, 0.0150
ш, 0.0078, 0.0058
щ, 0.0017, 0.0035
ъ, 0.0000, 0.0005
ы, 0.0121, 0.0168
ь, 0.0214, 0.0157
э, 0.0000, 0.0042
ю, 0.0072, 0.0048
я, 0.0528, 0.0018
ё, 0.0000, 0.0001

 The first column is more the consonant state,       
the second column is more the vowel state, 
but there is overlap


In [88]:
# 3 hidden states
M = len(set(obs))
N = 3
pi = np.random.dirichlet(np.ones(N))
A = np.random.dirichlet(np.ones(N), size=N).T
B = np.random.dirichlet(np.ones(M), size=N).T

# fit the hmm
hmm = HMM()
hmm.fit(obs, pi, A, B, max_iter=150)

In [92]:
for i in range(len(hmm.B)):
    print(u"{0}, {1:0.4f}, {2:0.4f}, {3:0.4f}"
                .format(symbols[i], hmm.B[i,0], hmm.B[i,1], hmm.B[i,2]))

 , 0.0432, 0.0978, 0.4208
а, 0.1965, 0.0000, 0.0002
б, 0.0000, 0.0327, 0.0102
в, 0.0000, 0.0716, 0.0464
г, 0.0000, 0.0372, 0.0140
д, 0.0010, 0.0453, 0.0214
е, 0.1534, 0.0000, 0.0490
ж, 0.0000, 0.0188, 0.0049
з, 0.0000, 0.0272, 0.0185
и, 0.1425, 0.0000, 0.0097
й, 0.0000, 0.0000, 0.0339
к, 0.0000, 0.0581, 0.0314
л, 0.0000, 0.0920, 0.0319
м, 0.0000, 0.0432, 0.0248
н, 0.0000, 0.1306, 0.0345
о, 0.2669, 0.0000, 0.0028
п, 0.0000, 0.0465, 0.0216
р, 0.0000, 0.0865, 0.0121
с, 0.0004, 0.0434, 0.0961
т, 0.0000, 0.1043, 0.0281
у, 0.0645, 0.0000, 0.0019
ф, 0.0000, 0.0021, 0.0016
х, 0.0000, 0.0122, 0.0077
ц, 0.0000, 0.0076, 0.0004
ч, 0.0000, 0.0202, 0.0148
ш, 0.0000, 0.0147, 0.0037
щ, 0.0000, 0.0075, 0.0000
ъ, 0.0006, 0.0002, 0.0000
ы, 0.0420, 0.0000, 0.0000
ь, 0.0499, 0.0000, 0.0000
э, 0.0000, 0.0000, 0.0099
ю, 0.0029, 0.0000, 0.0176
я, 0.0359, 0.0000, 0.0300
ё, 0.0001, 0.0000, 0.0000


The first column is the vowel state, the second column is the hard consonants state, 
and the third column is the soft consonants state')